In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import xgboost
import cv2
import imblearn

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
np.sum(df.isnull())

In [ ]:
X=df.drop('Outcome', axis=1)

In [ ]:
Y=df['Outcome']

In [ ]:
X

In [ ]:
Y

In [ ]:
X.corr()

In [ ]:
X.corr()[(X.corr()>=0.8) | (X.corr()<=-0.8)]

# OVERSAMPLING

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros=RandomOverSampler()
x_res,y_res=ros.fit_sample(X,Y)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x_res,y_res, random_state=42, test_size=0.2, stratify=y_res)

In [ ]:
print(len(x_train),len(x_test), end=' ')
print()

print(len(y_train),len(y_test),end=' ')

In [ ]:
for x in set(X):
    print('{}....{}'.format(x,len(X[X==x])))

In [ ]:
for y in set(Y):
    print('{}....{}'.format(y,len(Y[Y==y])))

In [ ]:
for x in set(x_res):
    print('{}....{}'.format(x,len(x_res[x_res==x])))

In [ ]:
for y in set(y_res):
    print('{}....{}'.format(y,len(y_res[y_res==y])))

In [ ]:
for x in set(x_train):
    print('{}....{}'.format(x,len(x_train[x_train==x])))

In [ ]:
for x in set(x_test):
    print('{}....{}'.format(x,len(x_test[x_test==x])))

In [ ]:
for y in set(y_train):
    print('{}....{}'.format(y,len(y_train[y_train==y])))

In [ ]:
for y in set(y_test):
    print('{}....{}'.format(y,len(y_test[y_test==y])))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
#lr=LogisticRegression()
#lr.fit(x_train, y_train)

from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
x_train_scaled=sc_x.fit_transform(x_train)


#svc=SVC(kernel='rbf', gamma=2, C=0.000000001)
svc=SVC()

In [ ]:
#from sklearn.model_selection import cross_val_predict

#y_probas= cross_val_predict(lr,x_train, y_train,cv=5,
#                              method='predict_proba')

In [ ]:
#precisions, recalls, thresholds= precision_recall_curve(y_train, y_probas[:,1])

In [ ]:
#px.line(x=recalls,y=precisions,labels=dict(x="Recall", y="Precision"))

In [ ]:
#from sklearn.metrics import roc_curve

#fpr, tpr, thresholds=roc_curve(y_train, y_probas[:,1])

In [ ]:
#plt.plot(fpr, tpr, "b:", label="Linear reg")
#plt.plot(fpr, tpr, linewidth=2, label='linear reg')
#plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
#plt.legend(loc="lower right")
#plt.show()

In [ ]:
#from sklearn.metrics import roc_auc_score
#roc_auc_score(y_train, y_probas[:,1])

In [ ]:
#from sklearn.metrics import precision_recall_curve

#precisions, recalls, thresholds= precision_recall_curve(y_train, y_scores[:,1])

In [ ]:
#plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
#plt.plot(thresholds, recalls[:-1], "g-", label="Recall")

In [ ]:
from sklearn.model_selection import GridSearchCV

#kernel='rbf', gamma=2, C=0.000000001

params={
    'kernel':['rbf'],
    'gamma':[1,2,3],
    'C':[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1,2,3,4,5,6,7,8,9,10,100]
}

#scoring={'precision':'precision', 'recall':'recall'}
#gs=GridSearchCV(svc, cv=5, param_grid=params, n_jobs=-1, 
#                scoring=scoring,
#               refit='precision')

gs=GridSearchCV(svc, cv=10, param_grid=params, n_jobs=-1, 
                scoring='f1')
gs.fit(x_train_scaled, y_train)

In [ ]:
#gamma=2, C=0.000000001

In [ ]:
print(gs.best_params_)
print(gs.best_score_)

In [ ]:
svc_best=SVC(C= float(gs.best_params_.get('C')), gamma= float(gs.best_params_.get('gamma')), 
             kernel= gs.best_params_.get('kernel'))

svc_best.fit(x_train_scaled, y_train)

print(float(gs.best_params_.get('C')))
print(float(gs.best_params_.get('gamma')))
print(str(gs.best_params_.get('kernel')))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score,f1_score


y_train_pred=svc_best.predict(x_train_scaled)
print(confusion_matrix(y_train,y_train_pred))
print()
print()
print(classification_report(y_train,y_train_pred))
print()
print()
print('Precision =',precision_score(y_train, y_train_pred))
print('Recall =',recall_score(y_train, y_train_pred))
print('f1-score =',f1_score(y_train, y_train_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.model_selection import cross_val_predict


#y_train_pred=svc_best.predict(x_train_scaled)
y_train_pred= cross_val_predict(svc_best, x_train_scaled, y_train,cv=10)
print(confusion_matrix(y_train,y_train_pred))
print()
print()
print(classification_report(y_train,y_train_pred))
print()
print()
print('Precision =',precision_score(y_train, y_train_pred))
print('Recall =',recall_score(y_train, y_train_pred))
print('f1-score =',f1_score(y_train, y_train_pred))

In [ ]:
y_test_pred=svc_best.predict(sc_x.transform(x_test))
print(confusion_matrix(y_test, y_test_pred))
print()
print()
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred))
print()
print('Precision =',precision_score(y_test, y_test_pred))
print('Recall =',recall_score(y_test, y_test_pred))
print('f1-score =',f1_score(y_test, y_test_pred))

# SMOTE

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from imblearn.combine import SMOTETomek

smt=SMOTETomek(random_state=42)
x_res,y_res= smt.fit_sample(X,Y)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x_res,y_res, random_state=42, test_size=0.2, stratify=y_res)

In [ ]:
print(len(x_train),len(x_test), end=' ')
print()

print(len(y_train),len(y_test),end=' ')

In [ ]:
for x in set(X):
    print('{}....{}'.format(x,len(X[X==x])))

In [ ]:
for y in set(Y):
    print('{}....{}'.format(y,len(Y[Y==y])))

In [ ]:
for x in set(x_res):
    print('{}....{}'.format(x,len(x_res[x_res==x])))

In [ ]:
for y in set(y_res):
    print('{}....{}'.format(y,len(y_res[y_res==y])))

In [ ]:
for x in set(x_train):
    print('{}....{}'.format(x,len(x_train[x_train==x])))

In [ ]:
for x in set(x_test):
    print('{}....{}'.format(x,len(x_test[x_test==x])))

In [ ]:
for y in set(y_train):
    print('{}....{}'.format(y,len(y_train[y_train==y])))

In [ ]:
for y in set(y_test):
    print('{}....{}'.format(y,len(y_test[y_test==y])))

In [ ]:
#lr=LogisticRegression()
#lr.fit(x_train, y_train)
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
x_train_scaled=sc_x.fit_transform(x_train)


#svc=SVC(kernel='rbf', gamma=1, C=0.001)
svc=SVC()

In [ ]:
from sklearn.model_selection import GridSearchCV

#kernel='rbf', gamma=2, C=0.000000001

params={
    'kernel':['rbf'],
    'gamma':[1,2,3],
    'C':[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1,2,3,4,5,6,7,8,9,10,100]
}

#scoring={'precision':'precision', 'recall':'recall'}
#gs=GridSearchCV(svc, cv=5, param_grid=params, n_jobs=-1, 
#                scoring=scoring,
#               refit='precision')


gs=GridSearchCV(svc, cv=10, param_grid=params, n_jobs=-1, 
                scoring='f1')

gs.fit(x_train_scaled, y_train)

In [ ]:
#gamma=1, C=0.001

In [ ]:
print(gs.best_params_)
print(gs.best_score_)

In [ ]:
svc_best=SVC(C= float(gs.best_params_.get('C')), gamma= float(gs.best_params_.get('gamma')), 
             kernel= str(gs.best_params_.get('kernel')))
svc_best.fit(x_train_scaled, y_train)

print(float(gs.best_params_.get('C')))
print(float(gs.best_params_.get('gamma')))
print(str(gs.best_params_.get('kernel')))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score,f1_score

y_train_pred=svc_best.predict(x_train_scaled)
print(confusion_matrix(y_train,y_train_pred))
print()
print()
print(classification_report(y_train,y_train_pred))
print()
print()
print('Precision =',precision_score(y_train, y_train_pred))
print('Recall =',recall_score(y_train, y_train_pred))
print('f1-score =',f1_score(y_train, y_train_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.model_selection import cross_val_predict

y_train_pred= cross_val_predict(svc_best, x_train_scaled, y_train,cv=10)
#y_train_pred=svc_best.predict(x_train_scaled)
print(confusion_matrix(y_train,y_train_pred))
print()
print()
print(classification_report(y_train,y_train_pred))
print()
print()
print('Precision =',precision_score(y_train, y_train_pred))
print('Recall =',recall_score(y_train, y_train_pred))
print('f1-score =',f1_score(y_train, y_train_pred))

In [ ]:
y_test_pred=svc_best.predict(sc_x.transform(x_test))
print(confusion_matrix(y_test, y_test_pred))
print()
print()
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred))
print()
print('Precision =',precision_score(y_test, y_test_pred))
print('Recall =',recall_score(y_test, y_test_pred))
print('f1-score =',f1_score(y_test, y_test_pred))

In [ ]:
cvres= gs.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(mean_score), params)
#cvres